In [1]:
import random
import pandas as pd
import numpy as np
data_X=pd.read_csv('./data/x.csv',encoding='utf-8')
data_y=pd.read_csv('./data/y.csv',encoding='utf-8')
index =list(data_X.index)
random.Random(0).shuffle(index)
num=int(len(index)/5*4)
x_train,y_train=data_X.iloc[index[:num],:].values,data_y.iloc[index[:num],:].values
x_test,y_test=data_X.iloc[index[num:],:].values,data_y.iloc[index[num:],:].values
def get_batch(x_train,y_train,batch_size):
    _index_in_epoch = 0
    for i in range(10000):
        start = _index_in_epoch
        _index_in_epoch += batch_size
        if _index_in_epoch > x_train.shape[0]:
            index = list(range(x_train.shape[0]))
            random.Random(0).shuffle(index)
            x_train = x_train[index]
            y_train = y_train[index]
            start = 0
            _index_in_epoch = batch_size
        end = _index_in_epoch
        batch_xs = x_train[start:end]
        batch_ys = y_train[start:end]
        yield batch_xs,batch_ys
def get_batch_x(x_train,batch_size):
    _index_in_epoch = 0
    while True:
        start = _index_in_epoch
        _index_in_epoch += batch_size
        if _index_in_epoch > x_train.shape[0]:
            index = list(range(x_train.shape[0]))
            random.Random(0).shuffle(index)
            x_train = x_train[index]
            start = 0
            _index_in_epoch = batch_size
        end = _index_in_epoch
        batch_xs = x_train[start:end]
        yield batch_xs

In [2]:
import tensorflow as tf
from Autoencoder_norm import Autoencoder
training_epochs = 2000
batch_size = 128
display_step = 1
corruption_level = 0.2
sparse_reg = 0

#
n_inputs = 2048
n_hidden = 400
n_hidden_2= 200
n_hidden_3= 100
n_hidden_out = 20
n_outputs = 10
lr = 0.001
is_training=tf.placeholder(tf.bool, name = "train")

in_keep_prob=tf.placeholder("float",name = "keep")
# define the autoencoder
ae = Autoencoder(n_layers=[n_inputs, n_hidden],
                          transfer_function = tf.nn.relu,
                          optimizer = tf.train.AdamOptimizer(learning_rate = lr),
                          in_keep_prob=in_keep_prob,is_training=is_training)
ae_2 = Autoencoder(n_layers=[n_hidden,n_hidden_2],
                          transfer_function = tf.nn.relu,
                          optimizer = tf.train.AdamOptimizer(learning_rate = lr),
                          in_keep_prob=in_keep_prob,is_training=is_training)
ae_3= Autoencoder(n_layers=[n_hidden_2,n_hidden_3],
                          transfer_function = tf.nn.relu,
                          optimizer = tf.train.AdamOptimizer(learning_rate = lr),
                          in_keep_prob=in_keep_prob,is_training=is_training)
ae_4= Autoencoder(n_layers=[n_hidden_3,n_hidden_out],
                          transfer_function = tf.nn.relu,
                          optimizer = tf.train.AdamOptimizer(learning_rate = lr),
                          in_keep_prob=in_keep_prob,is_training=is_training)
# define the output layer using softmax
x = tf.placeholder(tf.float32, [None,n_hidden_out])
W = tf.Variable(tf.zeros([n_hidden_out, n_outputs]))
b = tf.Variable(tf.zeros([n_outputs]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, n_outputs])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
train_step  = tf.train.AdamOptimizer(learning_rate = lr).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## define the output layer using softmax in the fine tuning step

'''
# Go through the two autoencoders
x_ft = tf.placeholder(tf.float32, [None, n_inputs])
h = x_ft
for layer in range(len(ae.n_layers) - 1):
    # h = tf.nn.dropout(h, ae.in_keep_prob)
    h = ae.transfer(
        tf.add(tf.matmul(h, ae.weights['encode'][layer]['w']),ae.weights['encode'][layer]['b']))
for layer in range(len(ae_2nd.n_layers) - 1):
    # h = tf.nn.dropout(h, ae_2nd.in_keep_prob)
    h = ae_2nd.transfer(
        tf.add(tf.matmul(h, ae_2nd.weights['encode'][layer]['w']),ae_2nd.weights['encode'][layer]['b']))
y_ft = tf.matmul(h, W) + b
cross_entropy_ft = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_ft, labels=y_))
train_step_ft  = tf.train.AdamOptimizer(learning_rate = lr).minimize(cross_entropy_ft)
correct_prediction = tf.equal(tf.argmax(y_ft, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
'''
## Initialization
init = tf.global_variables_initializer()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
def run_encoder(sess,ae_n,x_train,x_test,batch_size,epochs=2000):
    data_X=np.vstack((x_train,x_test))
    for i in range(epochs):        
        tmp_batch=get_batch_x(data_X,batch_size)
        batch_xs=next(tmp_batch)
        c,_= sess.run(ae_n.partial_fit(),feed_dict={ae_n.x: batch_xs, in_keep_prob:0.9,is_training:True})
        if i %500==0:
            print(f'time_{i} cost is {c}')
    encoder_train=sess.run(ae_n.transform(),feed_dict={ae_n.x:x_train, in_keep_prob:0.9,is_training:True})
    encoder_test=sess.run(ae_n.transform(),feed_dict={ae_n.x:x_test,in_keep_prob:0.9,is_training:True})
    return encoder_train,encoder_test

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    _index_in_epoch = 0
    for i in range(2000):
        # 清洗数据
        tmp_batch=get_batch(data_X.values,data_y.values,batch_size)
        batch_xs,batch_ys=next(tmp_batch)
        # 定义每次送入网络的数据
        h_ae1_out = sess.run(ae.transform(),feed_dict={ae.x: batch_xs, ae.keep_prob : ae.in_keep_prob})
        temp = ae_2nd.partial_fit()
        cost, opt = sess.run(temp,feed_dict={ae_2nd.x: h_ae1_out, ae_2nd.keep_prob : ae_2nd.in_keep_prob})
        if i % 500 == 0:
            print(f'time_{i} is {cost}')
    ae_test_cost = sess.run(ae.calc_total_cost(),feed_dict={ae.x:x_test, ae.keep_prob : 1.0})
    print("Total cost: " + str(ae_test_cost))
    print("************************Finish the autoencoder training******************************")
    # Training the softmax layer
    _index_in_epoch = 0
    for i in range(2000):
        tmp_batch=get_batch(x_train,y_train,batch_size)
        batch_xs,batch_ys=next(tmp_batch)
        h_ae1_out = sess.run(ae.transform(), feed_dict={ae.x: batch_xs, ae.keep_prob : 1.0})
        h_ae2_out = sess.run(ae_2nd.transform(), feed_dict={ae_2nd.x: h_ae1_out, ae_2nd.keep_prob : 1.0})
        sess.run(train_step, feed_dict={x: h_ae2_out, y_: batch_ys})
    print("************************Finish the softmax output layer training******************************")
    print("Test accuracy before fine-tune")
    sess.run(train_step, feed_dict={x: h_ae2_out, y_: batch_ys})
    print(sess.run(accuracy, feed_dict={x_ft:x_test, y_: y_test,ae.keep_prob: 1.0}))

In [5]:
#编码层分开优化
with tf.Session() as sess:
    sess.run(init)
    e1_train,e1_test=run_encoder(sess,ae,x_train,x_test,batch_size,epochs=10)
    print(ae_2.in_keep_prob)
    e2_train,e2_test=run_encoder(sess,ae_2,e1_train,e1_test,batch_size,epochs=500)
    e3_train,e3_test=run_encoder(sess,ae_3,e2_train,e2_test,batch_size,epochs=500)
    e_out_train,e_out_test=run_encoder(sess,ae_4,e3_train,e3_test,batch_size,epochs=500)
    print("************************Finish the autoencoder training******************************")
    # Training the softmax layer
    for i in range(2000):
        tmp_batch=get_batch(e_out_train,y_train,batch_size)
        batch_xs,batch_ys=next(tmp_batch)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    print("************************Finish the softmax output layer training******************************")
    print("Test accuracy before fine-tune")
    print(sess.run(accuracy, feed_dict={x:e_out_test, y_: y_test,ae.keep_prob: 1.0}))

time_0 cost is 31299.96484375
Tensor("keep:0", dtype=float32)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,2048]
	 [[node Placeholder (defined at E:\python\falut diagnosis\Fault_Diagnosis-master\file_autoencoding\Autoencoder_norm.py:18) ]]

Original stack trace for 'Placeholder':
  File "D:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\anaconda\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "D:\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "D:\anaconda\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "D:\anaconda\lib\asyncio\base_events.py", line 534, in run_forever
  

In [24]:
data_X.values.shape

(769, 2048)